In [1]:
#imports
import torch
from torch.utils.data import DataLoader
import MinkowskiEngine as ME
from tqdm import tqdm
import os
import time

import transforms as t
from dataset import SHREC2022Primitives
from networks import MinkowskiFCNN
from utils import minkowski_collate, create_input_batch 
import losses

# configuring paths for data and checkpoints
#data_path = "/home/beastmaster/Desktop/Eleftheria/shrec/dataset/" 
data_path = "/home/ioannis/Desktop/programming/data/SHREC/SHREC2022/dataset"
#checkpoint_path = "/home/beastmaster/Desktop/vlassisgiannis/SHREC2022_PrimitiveRecognition-regressors/checkpoints"
checkpoint_path = "/home/ioannis/Desktop/programming/phd/SHREC/SHREC2022/checkpoints"

cls_checkpoint = os.path.join(checkpoint_path, "classification.pth")
plane_checkpoint = os.path.join(checkpoint_path,"plane.pth")
sphere_checkpoint = os.path.join(checkpoint_path,"sphere_no_radius.pth")
cylinder_checkpoint = os.path.join(checkpoint_path,"cylinder.pth")
cone_checkpoint = os.path.join(checkpoint_path,"cone.pth")
torus_checkpoint = os.path.join(checkpoint_path,"torus.pth")

# General Parameters
valid_split=0.15


# Initializing transforms
# (will use the same augmentation for all networks and tasks)
train_transforms = [t.KeepInitialPoints(),
                    t.Translate(), 
                    t.SphereNormalization(), 
                    t.Initialization(),
                    t.RandomRotate(180, 0),
                    t.RandomRotate(180, 1),
                    t.RandomRotate(180, 2),
                    t.GaussianNoise(),
                    t.GetMean()]

valid_transforms = [t.KeepInitialPoints(),
                    t.Translate(), 
                    t.SphereNormalization(),
                    t.GetMean()]


# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cpu_device = torch.device("cpu")
print(f"Using Device: {device}")

Using Device: cuda


In [2]:
batch_size = 128

# network 
sphere_net = MinkowskiFCNN(in_channel=3,   # point features
                           out_channel=3,  # outputs only the center  
                             ).to(device)

# optimizer
optimizer = torch.optim.Adam(sphere_net.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 
                                            step_size=15,
                                            gamma=0.5)

sphere_loss = losses.SphereLoss()


# dataset 
t_dataset = SHREC2022Primitives(data_path, 
                                train=True, 
                                valid=False, 
                                valid_split=valid_split, 
                                transform=train_transforms,
                                category="sphere")

train_loader = DataLoader(t_dataset, 
                          batch_size=batch_size, 
                          shuffle=True, 
                          collate_fn=minkowski_collate, 
                          num_workers=8)

v_dataset = SHREC2022Primitives(data_path, 
                                train=True, 
                                valid=True, 
                                valid_split=valid_split, 
                                transform=valid_transforms,
                                category="sphere")

valid_loader = DataLoader(v_dataset, 
                          batch_size=1, 
                          shuffle=False, 
                          collate_fn=minkowski_collate, 
                          num_workers=8)

Dataset path:  /home/ioannis/Desktop/programming/data/SHREC/SHREC2022/dataset/training
Creating a new train-validation split.
Dataset path:  /home/ioannis/Desktop/programming/data/SHREC/SHREC2022/dataset/training
Specified split already exists. Using the existing one.


In [3]:
num_epochs = 100
eval_step = 1

start = time.time()

for epoch in range(num_epochs):
    # for loss and accuracy tracking the training set
    m_loss = 0
    m_center_loss = 0
    m_radius_loss = 0
    
    for batch in tqdm(train_loader):
        
        optimizer.zero_grad()
        
        # reading the data and formating them
        labels = batch["labels"].to(device)
        gt = labels[:,1:]
        minknet_input = create_input_batch(
            batch, 
            device=device,
            quantization_size=0.05
        )
        
        # activating network
        pred = sphere_net(minknet_input)       
        
        pred = torch.cat([torch.ones(pred.shape[0], 1, device=pred.device), pred ], dim=-1)
        
        # calculating losses
        c_loss, r_loss = sphere_loss(pred, gt, batch["trans"])
        c_loss = c_loss.mean(0)
                
        loss = c_loss
        
        loss.backward()
        optimizer.step()
        
        m_center_loss += c_loss.item()
        m_loss += loss.item()
    
    # stepping the scheduler
    scheduler.step()
    
    m_loss /= len(train_loader)
    m_center_loss /= len(train_loader)
    
    print(f" Epoch: {epoch} | Training: loss = {m_loss}")
    print(f" Center loss: {m_center_loss}")

    
    # validation
    if (epoch+1)%eval_step == 0:
        sphere_net.eval()
        with torch.no_grad():
            m_loss = 0
            m_center_loss = 0
            m_radius_loss = 0

            for batch in tqdm(valid_loader):
                
                transformed_points = torch.clone(batch['features']).to(device)
                
                # reading the data and formating them
                labels = batch["labels"].to(device)
                gt = labels[:,1:]
                minknet_input = create_input_batch(
                    batch, 
                    device=device,
                    quantization_size=0.05
                )

                # activating network
                pred = sphere_net(minknet_input)       
                
                
                radius = transformed_points - pred
                radius = (radius * radius).sum(-1).sqrt().mean(0)
                
                #print(radius.shape)
                #print(radius.unsqueeze(-1).shape)
                
                pred = torch.cat([radius.unsqueeze(-1).unsqueeze(-1), pred], dim=-1)
                
                # calculating losses
                c_loss, r_loss = sphere_loss(pred, gt, batch["trans"])
                c_loss = c_loss.mean(0)
                r_loss = r_loss.mean(0)

                loss = r_loss + c_loss

                m_center_loss += c_loss.item()
                m_radius_loss += r_loss.item()
                m_loss += loss.item()

            m_loss        /= len(valid_loader)
            m_center_loss /= len(valid_loader)
            m_radius_loss /= len(valid_loader)

            print(f" ------------ | Validation: loss = {m_loss}")
            print(f" Center loss: {m_center_loss} | Radius loss: {m_radius_loss}")
        
        
        sphere_net.train()
        
finish = time.time()

100%|██████████| 62/62 [00:20<00:00,  2.97it/s]


 Epoch: 0 | Training: loss = 4.344136926435655
 Center loss: 4.344136926435655


100%|██████████| 1380/1380 [00:16<00:00, 82.92it/s]


 ------------ | Validation: loss = 3.708247376030203
 Center loss: 2.8216318811050387 | Radius loss: 0.8866154892951392


100%|██████████| 62/62 [00:20<00:00,  3.08it/s]


...

 ------------ | Validation: loss = 0.09649243053969732
 Center loss: 0.06111031416876885 | Radius loss: 0.03538211668665725


100%|██████████| 62/62 [00:21<00:00,  2.86it/s]


 Epoch: 99 | Training: loss = 0.11889146676947994
 Center loss: 0.11889146676947994


100%|██████████| 1380/1380 [00:17<00:00, 79.06it/s]

 ------------ | Validation: loss = 0.09633313022930688
 Center loss: 0.061373527929756984 | Radius loss: 0.034959602174916934


In [4]:
torch.save(sphere_net.state_dict(), sphere_checkpoint)
# sending back to cpu to free memory
sphere_net.to(cpu_device)
# time statistics
print(f"Time to train sphere regressor network: {finish - start} sec")

Time to train sphere regressor network: 3737.816478252411 sec
